<img src="img/mmselfsup_logo.png">

# 在 MMDetection 中使用自监督预训练模型

<a href="https://colab.research.google.com/github/wangruohui/OpenMMLabCourse/blob/main/codes/MMSelfSup_tutorials/【6】在%20MMDetection%20中使用自监督预训练模型.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MMDetection Repo**：[https://github.com/open-mmlab/mmdetection](https://github.com/open-mmlab/mmdetection)

**MMDetection 官方文档链接**：[https://mmdetection.readthedocs.io/en/latest](https://mmdetection.readthedocs.io/en/latest)

**MMSelfSup Repo**：[https://github.com/open-mmlab/mmselfsup](https://github.com/open-mmlab/mmselfsup)

**MMSelfSup 官方文档链接**：[https://mmselfsup.readthedocs.io/en/latest](https://mmselfsup.readthedocs.io/en/latest)



**作者**：OpenMMLab

## 0. 任务介绍

我们除了可以直接使用 MMSelfSup 里 `tools/benchmark` 底下的工具对自监督预训练模型进行评估，还可以直接将保存好的模型文件用于 OpenMMLab 的其他库（如 MMDetection、MMSegmentation）中进行训练。

本教程将演示：如何在 MMDetection 中使用自监督预训练模型的权值进行目标检测任务的训练。

**注意：下游任务的 backbone 必须和自监督预训练模型的 backbone 保持一致，才能进行自监督预训练权值的迁移！**

## 1. 环境配置

### 1.1 查看 Python、PyTorch 和 Torchvision 的版本

In [ ]:
# Check nvcc version
!nvcc -V

In [ ]:
# Check GCC version
!gcc --version

In [ ]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

### 1.2 安装 MMDetection 的依赖库：MMCV

In [ ]:
!pip install -U openmim
!mim install mmcv

### 1.3  安装 MMDetection
其他方式的安装详情请参考： [MMDetection 安装文档](https://mmdetection.readthedocs.io/en/latest/get_started.html#best-practices)。

In [ ]:
%cd /content

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -v -e .

### 1.5 检查安装是否正确

In [ ]:
import mmdet
print(mmdet.__version__)

## 2. 准备数据集

### 2.0 数据集介绍

本教程将在 `COCO2017` 数据集上训练目标检测任务。

COCO 是一个大规模目标检测、图像分割和图像描述数据集。它包含 80 个物体类别，150 万个物体实例，约 33 万张图像（其中超过 20 万张图像有标注）。

数据集官方网址：https://cocodataset.org

### 2.1 下载并解压数据集

使用该命令可以下载并解压 COCO 数据集。该命令支持下载 COCO，VOC 和 LVIS 数据集。
```shell
python tools/misc/download_dataset.py --dataset-name ${DATASET_NAME} --unzip --delete
```

参数：
+ DATASET_NAME：支持数据集的名称 `coco2017`，`voc2007` 和 `lvis`

In [ ]:
%cd /content/mmdetection
!mkdir -p data/coco
%cd data/coco

In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip
!unzip train2017.zip
!rm -rf train2017.zip

!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip
!rm -rf val2017.zip

!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip
!rm -rf annotations_trainval2017.zip

## 3. 准备自监督预训练模型的 backbone 权值文件

### 3.1 针对自监督预训练过程中保存的 ckeckpoint 文件

可以参考前面的教程，使用命令 `tools/model_converters/extract_backbone_weights.py` 来提取自监督预训练模型的 backbone 权值，代码如下：

```python
python tools/model_converters/extract_backbone_weights.py {CHECKPOINT} {MODEL_FILE}
```

参数:
- CHECKPOINT：自监督预训练过程中保存下来（名为 `epoch_*.pth`）的模型文件路径
- MODEL_FILE：输出 backbone 权重文件的保存路径。

### 3.2 针对 MMSelfSup 模型库里的模型文件

**注意：MMSelfSup 的 [模型库](https://github.com/open-mmlab/mmselfsup/blob/master/docs/en/model_zoo.md) 中的模型文件都已经提取过 backbone 权值，不需要再次提取！**我们直接使用即可。

在模型库中找到在 SimCLR 的预训练模型文件 `simclr_resnet50_8xb32-coslr-200e_in1k`，下载放在 `checkpoints` 文件夹里 

In [ ]:
%cd /content/mmdetection
!mkdir checkpoints
!wget https://download.openmmlab.com/mmselfsup/simclr/simclr_resnet50_8xb32-coslr-200e_in1k_20220428-46ef6bb9.pth -P checkpoints

## 4. 使用自监督预训练模型的权值初始化目标检测的模型

### 4.1 写目标检测任务的配置文件

1. 新建一个名为 `faster_rcnn_simclr-pretrained_r50_caffe_fpn_9k_coco.py` 的配置文件。


2. `faster_rcnn_simclr-pretrained_r50_caffe_fpn_9k_coco.py` 训练配置文件的内容：
    1. 继承 [faster_rcnn_r50_caffe_fpn_90k_coco.py](https://github.com/open-mmlab/mmdetection/blob/master/configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_90k_coco.py) 配置文件
    2. 将模型的 checkpoint 参数修改为自监督预训练模型的 backbone 所在路径
    3. 根据 batch size 调整学习率（调整原则请参考：[这里](https://mmselfsup.readthedocs.io/zh_CN/latest/get_started.html#id2)）
    4. 修改总共训练的迭代次数 max_iters、模型训练多少个 iteration 评估一次、模型训练多少个 iteration 保存一次 checkpoint 文件等参数

In [ ]:
%%writefile /content/mmdetection/configs/faster_rcnn/faster_rcnn_simclr-pretrained_r50_caffe_fpn_9k_coco.py
_base_ = 'faster_rcnn_r50_caffe_fpn_90k_coco.py'

model = dict(
    backbone=dict(
        frozen_stages=-1,
        init_cfg=dict(
            type='Pretrained',
            checkpoint='checkpoints/simclr_resnet50_8xb32-coslr-200e_in1k_20220428-46ef6bb9.pth')
    )
)

# optimizer
optimizer = dict(
    lr=0.02 * (1 / 8)
)

# Runner type
runner = dict(_delete_=True, type='IterBasedRunner', max_iters=9000)

checkpoint_config = dict(interval=3000)

evaluation = dict(interval=3000)

### 4.2 开始训练目标检测模型

在训练目标检测模型时，我们使用 [tools/train.py](https://github.com/open-mmlab/mmdetection/blob/master/tools/dist_train.sh) 训练工具来启动训练。

```shell
python tools/train.py \
    ${CONFIG_FILE} \
    [optional arguments]
```

参数：
- CONFIG_FILE：“检测” 评估所使用的配置文件，位于 configs 里对应模型的目录下

详情请参考：[文档](https://mmdetection.readthedocs.io/en/latest/1_exist_data_model.html#training-on-a-single-gpu)

In [ ]:
%cd /content/mmdetection

In [ ]:
!python tools/train.py \
configs/faster_rcnn/faster_rcnn_simclr-pretrained_r50_caffe_fpn_9k_coco.py